In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [2]:
churn_data= pd.read_csv('D:\Education\Courses and Certies\Machine Learning Kaggle\LogisticReg\LogisticReg\churn_data.csv')

In [3]:
cust_data= pd.read_csv('D:\Education\Courses and Certies\Machine Learning Kaggle\LogisticReg\LogisticReg\customer_data.csv')

In [4]:
int_data= pd.read_csv('D:\Education\Courses and Certies\Machine Learning Kaggle\LogisticReg\LogisticReg\internet_data.csv')

In [5]:
#mergind all three datasets into final Telecom dataset
data1= pd.merge(churn_data,cust_data, how='left',left_on='customerID',right_on='customerID')

In [6]:
telecom= pd.merge(data1,int_data, how='left',left_on='customerID',right_on='customerID')

In [7]:
telecom.shape

(7043, 21)

## Replacing values of Yes and No by 1 and 0

In [8]:
telecom['PaperlessBilling']= telecom['PaperlessBilling'].replace(['No','Yes'],[0,1])
telecom['Dependents']= telecom['Dependents'].replace(['No','Yes'],[0,1])
telecom['OnlineSecurity']= telecom['OnlineSecurity'].replace(['No','Yes'],[0,1])
telecom['OnlineBackup']= telecom['OnlineBackup'].replace(['No','Yes'],[0,1])
telecom['DeviceProtection']= telecom['DeviceProtection'].replace(['No','Yes'],[0,1])
telecom['TechSupport']= telecom['TechSupport'].replace(['No','Yes'],[0,1])
telecom['StreamingTV']= telecom['StreamingTV'].replace(['No','Yes'],[0,1])
telecom['StreamingMovies']= telecom['StreamingMovies'].replace(['No','Yes'],[0,1])
telecom['PhoneService']= telecom['PhoneService'].replace(['No','Yes'],[0,1])
telecom['Churn']= telecom['Churn'].replace(['No','Yes'],[0,1])
telecom['Partner']= telecom['Partner'].replace(['No','Yes'],[0,1])

In [9]:
telecom['PhoneService'].astype(float)
telecom['PaperlessBilling'].astype(int)
telecom['MonthlyCharges'].astype(int)
telecom['Dependents'].astype(int)
# telecom['TotalCharges'].astype(float)
# telecom['OnlineSecurity'].astype(int)
# telecom['OnlineBackup'].astype(int)
# telecom['DeviceProtection'].astype(int)
# telecom['TechSupport'].astype(int)
# telecom['StreamingTV'].astype(int)
# telecom['StreamingMovies'].astype(int)
telecom.head()

,customerID,tenure,PhoneService,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,gender,...,Partner,Dependents,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
0,7590-VHVEG,1,0,Month-to-month,1,Electronic check,29.85,29.85,0,Female,...,1,0,No phone service,DSL,0,1,0,0,0,0
1,5575-GNVDE,34,1,One year,0,Mailed check,56.95,1889.5,0,Male,...,0,0,No,DSL,1,0,1,0,0,0
2,3668-QPYBK,2,1,Month-to-month,1,Mailed check,53.85,108.15,1,Male,...,0,0,No,DSL,1,1,0,0,0,0
3,7795-CFOCW,45,0,One year,0,Bank transfer (automatic),42.30,1840.75,0,Male,...,0,0,No phone service,DSL,1,0,1,1,0,0
4,9237-HQITU,2,1,Month-to-month,1,Electronic check,70.70,151.65,1,Female,...,0,0,No,Fiber optic,0,0,0,0,0,0


In [10]:
telecom[telecom['TotalCharges']==' ']
#11 rows are null

,customerID,tenure,PhoneService,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,gender,...,Partner,Dependents,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
488,4472-LVYGI,0,0,Two year,1,Bank transfer (automatic),52.55,,0,Female,...,1,1,No phone service,DSL,1,0,1,1,1,0
753,3115-CZMZD,0,1,Two year,0,Mailed check,20.25,,0,Male,...,0,1,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service
936,5709-LVOEQ,0,1,Two year,0,Mailed check,80.85,,0,Female,...,1,1,No,DSL,1,1,1,0,1,1
1082,4367-NUYAO,0,1,Two year,0,Mailed check,25.75,,0,Male,...,1,1,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service
1340,1371-DWPAZ,0,0,Two year,0,Credit card (automatic),56.05,,0,Female,...,1,1,No phone service,DSL,1,1,1,1,1,0
3331,7644-OMVMY,0,1,Two year,0,Mailed check,19.85,,0,Male,...,1,1,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service
3826,3213-VVOLG,0,1,Two year,0,Mailed check,25.35,,0,Male,...,1,1,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service
4380,2520-SGTTA,0,1,Two year,0,Mailed check,20.00,,0,Female,...,1,1,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service
5218,2923-ARZLG,0,1,One year,1,Mailed check,19.70,,0,Male,...,1,1,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service
6670,4075-WKNIU,0,1,Two year,0,Mailed check,73.35,,0,Female,...,1,1,Yes,DSL,0,1,1,1,1,0


In [11]:
telecom['MultipleLines'].value_counts()

No                  3390
Yes                 2971
No phone service     682
Name: MultipleLines, dtype: int64

In [12]:
telecom['InternetService'].value_counts()

Fiber optic    3096
DSL            2421
No             1526
Name: InternetService, dtype: int64

In [13]:
telecom['OnlineSecurity'].value_counts()

0                      3498
1                      2019
No internet service    1526
Name: OnlineSecurity, dtype: int64

In [14]:
telecom['OnlineBackup'].value_counts()

0                      3088
1                      2429
No internet service    1526
Name: OnlineBackup, dtype: int64

In [15]:
telecom['DeviceProtection'].value_counts()

0                      3095
1                      2422
No internet service    1526
Name: DeviceProtection, dtype: int64

In [16]:
telecom['TechSupport'].value_counts()

0                      3473
1                      2044
No internet service    1526
Name: TechSupport, dtype: int64

In [17]:
telecom['StreamingTV'].value_counts()

0                      2810
1                      2707
No internet service    1526
Name: StreamingTV, dtype: int64

In [18]:
telecom['StreamingMovies'].value_counts()

0                      2785
1                      2732
No internet service    1526
Name: StreamingMovies, dtype: int64

In [19]:
telecom['OnlineSecurity'].value_counts()

0                      3498
1                      2019
No internet service    1526
Name: OnlineSecurity, dtype: int64

In [20]:
telecom.columns

Index(['customerID', 'tenure', 'PhoneService', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn', 'gender',
       'SeniorCitizen', 'Partner', 'Dependents', 'MultipleLines',
       'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
       'TechSupport', 'StreamingTV', 'StreamingMovies'],
      dtype='object')

## Creating dummies for categorical variables 

In [21]:
telecom = pd.get_dummies(telecom, columns = ['Contract'], prefix='Contract', drop_first=True )
telecom = pd.get_dummies(telecom, columns = ['PaymentMethod'], prefix='PaymentMethod', drop_first=True )
telecom = pd.get_dummies(telecom, columns = ['gender'], prefix='gender', drop_first= True)

In [22]:
telecom = pd.get_dummies(telecom, columns = ['MultipleLines'], prefix='MultipleLines', drop_first=True )
telecom = pd.get_dummies(telecom, columns = ['InternetService'], prefix='InternetService', drop_first= True)

In [23]:
telecom = pd.get_dummies(telecom, columns = ['OnlineBackup'], prefix='OnlineBackup', drop_first=True )
telecom = pd.get_dummies(telecom, columns = ['DeviceProtection'], prefix='DeviceProtection', drop_first= True)
telecom = pd.get_dummies(telecom, columns = ['TechSupport'], prefix='TechSupport', drop_first=True )
telecom = pd.get_dummies(telecom, columns = ['StreamingTV'], prefix='StreaminTV', drop_first= True)
telecom = pd.get_dummies(telecom, columns = ['StreamingMovies'], prefix='StreaminMovies', drop_first= True)

In [24]:
telecom = pd.get_dummies(telecom, columns = ['OnlineSecurity'], prefix='OnlineSecurity', drop_first= True)

In [25]:
telecom.columns
# Index(['customerID', 'tenure', 'PhoneService', 'Contract', 'PaperlessBilling',
#        'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn', 'gender',
#        'SeniorCitizen', 'Partner', 'Dependents', 'MultipleLines',
#        'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
#        'TechSupport', 'StreamingTV', 'StreamingMovies'],
#       dtype='object')

Index(['customerID', 'tenure', 'PhoneService', 'PaperlessBilling',
       'MonthlyCharges', 'TotalCharges', 'Churn', 'SeniorCitizen', 'Partner',
       'Dependents', 'Contract_One year', 'Contract_Two year',
       'PaymentMethod_Credit card (automatic)',
       'PaymentMethod_Electronic check', 'PaymentMethod_Mailed check',
       'gender_Male', 'MultipleLines_No phone service', 'MultipleLines_Yes',
       'InternetService_Fiber optic', 'InternetService_No', 'OnlineBackup_1',
       'OnlineBackup_No internet service', 'DeviceProtection_1',
       'DeviceProtection_No internet service', 'TechSupport_1',
       'TechSupport_No internet service', 'StreaminTV_1',
       'StreaminTV_No internet service', 'StreaminMovies_1',
       'StreaminMovies_No internet service', 'OnlineSecurity_1',
       'OnlineSecurity_No internet service'],
      dtype='object')

In [26]:
#This will drop whole row if any value is null value
telecom.dropna(inplace=True)

In [27]:
#some values are empty string and have to remove them
telecom= telecom[telecom.TotalCharges != ' ']

In [28]:
telecom['TotalCharges'].astype(float)

0         29.85
1       1889.50
2        108.15
3       1840.75
4        151.65
         ...   
7038    1990.50
7039    7362.90
7040     346.45
7041     306.60
7042    6844.50
Name: TotalCharges, Length: 7032, dtype: float64

In [29]:
#removing 11 rows having no totalcharges values
telecom.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7032 entries, 0 to 7042
Data columns (total 32 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   customerID                             7032 non-null   object 
 1   tenure                                 7032 non-null   int64  
 2   PhoneService                           7032 non-null   int64  
 3   PaperlessBilling                       7032 non-null   int64  
 4   MonthlyCharges                         7032 non-null   float64
 5   TotalCharges                           7032 non-null   object 
 6   Churn                                  7032 non-null   int64  
 7   SeniorCitizen                          7032 non-null   int64  
 8   Partner                                7032 non-null   int64  
 9   Dependents                             7032 non-null   int64  
 10  Contract_One year                      7032 non-null   uint8  
 11  Cont

In [30]:
# #Finding the IQR value for the dataframe
# Q1 = telecom.quantile(0.25)
# Q3 = telecom.quantile(0.75)
# IQR = Q3 - Q1
# print(IQR)

In [31]:
telecom

,customerID,tenure,PhoneService,PaperlessBilling,MonthlyCharges,TotalCharges,Churn,SeniorCitizen,Partner,Dependents,...,DeviceProtection_1,DeviceProtection_No internet service,TechSupport_1,TechSupport_No internet service,StreaminTV_1,StreaminTV_No internet service,StreaminMovies_1,StreaminMovies_No internet service,OnlineSecurity_1,OnlineSecurity_No internet service
0,7590-VHVEG,1,0,1,29.85,29.85,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,5575-GNVDE,34,1,0,56.95,1889.5,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,3668-QPYBK,2,1,1,53.85,108.15,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,7795-CFOCW,45,0,0,42.30,1840.75,0,0,0,0,...,1,0,1,0,0,0,0,0,1,0
4,9237-HQITU,2,1,1,70.70,151.65,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,24,1,1,84.80,1990.5,0,0,1,1,...,1,0,1,0,1,0,1,0,1,0
7039,2234-XADUH,72,1,1,103.20,7362.9,0,0,1,1,...,1,0,0,0,1,0,1,0,0,0
7040,4801-JZAZL,11,0,1,29.60,346.45,0,0,1,1,...,0,0,0,0,0,0,0,0,1,0
7041,8361-LTMKD,4,1,1,74.40,306.6,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
#finding null value in columns
telecom.isnull().sum()

customerID                               0
tenure                                   0
PhoneService                             0
PaperlessBilling                         0
MonthlyCharges                           0
TotalCharges                             0
Churn                                    0
SeniorCitizen                            0
Partner                                  0
Dependents                               0
Contract_One year                        0
Contract_Two year                        0
PaymentMethod_Credit card (automatic)    0
PaymentMethod_Electronic check           0
PaymentMethod_Mailed check               0
gender_Male                              0
MultipleLines_No phone service           0
MultipleLines_Yes                        0
InternetService_Fiber optic              0
InternetService_No                       0
OnlineBackup_1                           0
OnlineBackup_No internet service         0
DeviceProtection_1                       0
DeviceProte

## Normalising continous features
## here we have normalised only those features that are continous values and we can check by df.describe and taking only those variables

In [33]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
num_vars = ['tenure', 'MonthlyCharges', 'TotalCharges']
telecom[num_vars] = scaler.fit_transform(telecom[num_vars])

In [34]:
telecom

,customerID,tenure,PhoneService,PaperlessBilling,MonthlyCharges,TotalCharges,Churn,SeniorCitizen,Partner,Dependents,...,DeviceProtection_1,DeviceProtection_No internet service,TechSupport_1,TechSupport_No internet service,StreaminTV_1,StreaminTV_No internet service,StreaminMovies_1,StreaminMovies_No internet service,OnlineSecurity_1,OnlineSecurity_No internet service
0,7590-VHVEG,0.000000,0,1,0.115423,0.001275,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,5575-GNVDE,0.464789,1,0,0.385075,0.215867,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,3668-QPYBK,0.014085,1,1,0.354229,0.010310,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,7795-CFOCW,0.619718,0,0,0.239303,0.210241,0,0,0,0,...,1,0,1,0,0,0,0,0,1,0
4,9237-HQITU,0.014085,1,1,0.521891,0.015330,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,0.323944,1,1,0.662189,0.227521,0,0,1,1,...,1,0,1,0,1,0,1,0,1,0
7039,2234-XADUH,1.000000,1,1,0.845274,0.847461,0,0,1,1,...,1,0,0,0,1,0,1,0,0,0
7040,4801-JZAZL,0.140845,0,1,0.112935,0.037809,0,0,1,1,...,0,0,0,0,0,0,0,0,1,0
7041,8361-LTMKD,0.042254,1,1,0.558706,0.033210,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
#Percentage of people who have churn

len(telecom[telecom['Churn']==1])/len(telecom)*100

26.578498293515356

In [36]:
telecom.columns

Index(['customerID', 'tenure', 'PhoneService', 'PaperlessBilling',
       'MonthlyCharges', 'TotalCharges', 'Churn', 'SeniorCitizen', 'Partner',
       'Dependents', 'Contract_One year', 'Contract_Two year',
       'PaymentMethod_Credit card (automatic)',
       'PaymentMethod_Electronic check', 'PaymentMethod_Mailed check',
       'gender_Male', 'MultipleLines_No phone service', 'MultipleLines_Yes',
       'InternetService_Fiber optic', 'InternetService_No', 'OnlineBackup_1',
       'OnlineBackup_No internet service', 'DeviceProtection_1',
       'DeviceProtection_No internet service', 'TechSupport_1',
       'TechSupport_No internet service', 'StreaminTV_1',
       'StreaminTV_No internet service', 'StreaminMovies_1',
       'StreaminMovies_No internet service', 'OnlineSecurity_1',
       'OnlineSecurity_No internet service'],
      dtype='object')

In [37]:
l= ['Contract_One year', 'Contract_Two year',
       'PaymentMethod_Credit card (automatic)',
       'PaymentMethod_Electronic check', 'PaymentMethod_Mailed check',
       'gender_Male', 'MultipleLines_No phone service', 'MultipleLines_Yes',
       'InternetService_Fiber optic', 'InternetService_No', 'OnlineBackup_1',
       'OnlineBackup_No internet service', 'DeviceProtection_1',
       'DeviceProtection_No internet service', 'TechSupport_1',
       'TechSupport_No internet service', 'StreaminTV_1',
       'StreaminTV_No internet service', 'StreaminMovies_1',
       'StreaminMovies_No internet service', 'OnlineSecurity_1',
       'OnlineSecurity_No internet service']

In [38]:
for i in l:
    telecom[i]= telecom[i].astype(int)

In [39]:
telecom.dtypes

customerID                                object
tenure                                   float64
PhoneService                               int64
PaperlessBilling                           int64
MonthlyCharges                           float64
TotalCharges                             float64
Churn                                      int64
SeniorCitizen                              int64
Partner                                    int64
Dependents                                 int64
Contract_One year                          int32
Contract_Two year                          int32
PaymentMethod_Credit card (automatic)      int32
PaymentMethod_Electronic check             int32
PaymentMethod_Mailed check                 int32
gender_Male                                int32
MultipleLines_No phone service             int32
MultipleLines_Yes                          int32
InternetService_Fiber optic                int32
InternetService_No                         int32
OnlineBackup_1      

### Splitting the dataset into train and test

In [40]:
#4922.4

In [41]:
telecom.columns

Index(['customerID', 'tenure', 'PhoneService', 'PaperlessBilling',
       'MonthlyCharges', 'TotalCharges', 'Churn', 'SeniorCitizen', 'Partner',
       'Dependents', 'Contract_One year', 'Contract_Two year',
       'PaymentMethod_Credit card (automatic)',
       'PaymentMethod_Electronic check', 'PaymentMethod_Mailed check',
       'gender_Male', 'MultipleLines_No phone service', 'MultipleLines_Yes',
       'InternetService_Fiber optic', 'InternetService_No', 'OnlineBackup_1',
       'OnlineBackup_No internet service', 'DeviceProtection_1',
       'DeviceProtection_No internet service', 'TechSupport_1',
       'TechSupport_No internet service', 'StreaminTV_1',
       'StreaminTV_No internet service', 'StreaminMovies_1',
       'StreaminMovies_No internet service', 'OnlineSecurity_1',
       'OnlineSecurity_No internet service'],
      dtype='object')

In [42]:
train= telecom.iloc[:4922,]
test=telecom.iloc[4922:,]

In [43]:
train_x=train[['tenure', 'PhoneService', 'PaperlessBilling',
       'MonthlyCharges', 'TotalCharges', 'SeniorCitizen', 'Partner',
       'Dependents', 'Contract_One year', 'Contract_Two year',
       'PaymentMethod_Credit card (automatic)',
       'PaymentMethod_Electronic check', 'PaymentMethod_Mailed check',
       'gender_Male', 'MultipleLines_No phone service', 'MultipleLines_Yes',
       'InternetService_Fiber optic', 'InternetService_No', 'OnlineBackup_1',
       'OnlineBackup_No internet service', 'DeviceProtection_1',
       'DeviceProtection_No internet service', 'TechSupport_1',
       'TechSupport_No internet service', 'StreaminTV_1',
       'StreaminTV_No internet service', 'StreaminMovies_1',
       'StreaminMovies_No internet service', 'OnlineSecurity_1',
       'OnlineSecurity_No internet service']]
test_x=test[[ 'tenure', 'PhoneService', 'PaperlessBilling',
       'MonthlyCharges', 'TotalCharges','SeniorCitizen', 'Partner',
       'Dependents', 'Contract_One year', 'Contract_Two year',
       'PaymentMethod_Credit card (automatic)',
       'PaymentMethod_Electronic check', 'PaymentMethod_Mailed check',
       'gender_Male', 'MultipleLines_No phone service', 'MultipleLines_Yes',
       'InternetService_Fiber optic', 'InternetService_No', 'OnlineBackup_1',
       'OnlineBackup_No internet service', 'DeviceProtection_1',
       'DeviceProtection_No internet service', 'TechSupport_1',
       'TechSupport_No internet service', 'StreaminTV_1',
       'StreaminTV_No internet service', 'StreaminMovies_1',
       'StreaminMovies_No internet service', 'OnlineSecurity_1',
       'OnlineSecurity_No internet service']]

train_y= train['Churn']

test_y= test['Churn']

## Here we will use sm.GLM

In [44]:
import statsmodels.api as sm

X_train = sm.add_constant(train_x)
mod= sm.GLM(train_y,(X_train),family= sm.families.Binomial()).fit() #here we have used binomial because we have only 0 and 1 output
print(mod.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  Churn   No. Observations:                 4922
Model:                            GLM   Df Residuals:                     4898
Model Family:                Binomial   Df Model:                           23
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -2005.1
Date:                Sat, 25 Sep 2021   Deviance:                       4010.3
Time:                        22:04:22   Pearson chi2:                 5.77e+03
No. Iterations:                     9                                         
Covariance Type:            nonrobust                                         
                                            coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------

In [45]:
# Using correlation matrix to filter the IV to use

In [46]:
# plt.figure(figsize=(20,10))
# sns.pairplot(telecom)

In [47]:
# plt.figure(figsize=(20,10))
# sns.heatmap(telecom.corr())

## Instead of checking P value and Correleation matrix to get the features, we can use the RFE to select the features

In [48]:
#HEre we are using RFE as the parameter selection from correlation matrix is very difficult. Here we will chose 13 parameters to fit them.
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
logreg= LogisticRegression()

rfe= RFE(logreg,15)
rfe= rfe.fit(train_x,train_y)
print(rfe.support_)
print(rfe.ranking_)

[ True  True False  True  True False False False  True  True False  True
 False False False  True  True  True False  True False  True  True False
 False False  True False  True False]
[ 1  1  3  1  1  6 15 10  1  1 11  1 16 14  8  1  1  1 12  1 13  1  1  4
  2  7  1  5  1  9]


In [49]:
# 'tenure', 'PhoneService', 'PaperlessBilling',
#        'MonthlyCharges', 'TotalCharges', 'SeniorCitizen', 'Partner',
#        'Dependents', 'Contract_One year', 'Contract_Two year',
#        'PaymentMethod_Credit card (automatic)',
#        'PaymentMethod_Electronic check', 'PaymentMethod_Mailed check',
#        'gender_Male', 'MultipleLines_No phone service', 'MultipleLines_Yes',
#        'InternetService_Fiber optic', 'InternetService_No', 'OnlineBackup_1',
#        'OnlineBackup_No internet service', 'DeviceProtection_1',
#        'DeviceProtection_No internet service', 'TechSupport_1',
#        'TechSupport_No internet service', 'StreaminTV_1',
#        'StreaminTV_No internet service', 'StreaminMovies_1',
#        'StreaminMovies_No internet service', 'OnlineSecurity_1',
#        'OnlineSecurity_No internet service'

In [50]:
train_x.columns[rfe.support_]

Index(['tenure', 'PhoneService', 'MonthlyCharges', 'TotalCharges',
       'Contract_One year', 'Contract_Two year',
       'PaymentMethod_Electronic check', 'MultipleLines_Yes',
       'InternetService_Fiber optic', 'InternetService_No',
       'OnlineBackup_No internet service',
       'DeviceProtection_No internet service', 'TechSupport_1',
       'StreaminMovies_1', 'OnlineSecurity_1'],
      dtype='object')

In [51]:
train_x=train[['tenure', 'PhoneService', 'TotalCharges',
       'Contract_One year', 'Contract_Two year',
       'PaymentMethod_Electronic check', 'MultipleLines_Yes',
       'InternetService_Fiber optic', 'InternetService_No',
       'OnlineBackup_No internet service',
       'DeviceProtection_No internet service', 'TechSupport_1',
        'OnlineSecurity_1']]
test_x=test[['tenure', 'PhoneService',  'TotalCharges',
       'Contract_One year', 'Contract_Two year',
       'PaymentMethod_Electronic check', 'MultipleLines_Yes',
       'InternetService_Fiber optic', 'InternetService_No',
       'OnlineBackup_No internet service',
       'DeviceProtection_No internet service', 'TechSupport_1',
       'OnlineSecurity_1']]

train_y= train['Churn']

test_y= test['Churn']

## Model building using SM.GLS model

In [52]:
import statsmodels.api as sm

X_train = sm.add_constant(train_x)
mod= sm.GLM(train_y,(X_train),family= sm.families.Binomial()).fit() #here we have used binomial because we have only 0 and 1 output
print(mod.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  Churn   No. Observations:                 4922
Model:                            GLM   Df Residuals:                     4910
Model Family:                Binomial   Df Model:                           11
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -2026.7
Date:                Sat, 25 Sep 2021   Deviance:                       4053.4
Time:                        22:04:23   Pearson chi2:                 6.03e+03
No. Iterations:                     7                                         
Covariance Type:            nonrobust                                         
                                           coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------


## Building model with the selected features

### Model building using Logistic regression from sklearn

In [53]:
from sklearn.linear_model import LogisticRegression

In [54]:
logsk= LogisticRegression()
m = logsk.fit(train_x,train_y)
print(m)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)


## Comparing LogisticRegression with the Statsmodels.GLS model

In [55]:
from sklearn.linear_model import LogisticRegression

logsk= LogisticRegression()
m = logsk.fit(train_x,train_y)
print(m)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)


In [56]:
import statsmodels.api as sm

X_train = sm.add_constant(train_x)
mod= sm.GLM(train_y,(X_train),family= sm.families.Binomial()).fit() #here we have used binomial because we have only 0 and 1 output
print(mod.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  Churn   No. Observations:                 4922
Model:                            GLM   Df Residuals:                     4910
Model Family:                Binomial   Df Model:                           11
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -2026.7
Date:                Sat, 25 Sep 2021   Deviance:                       4053.4
Time:                        22:04:23   Pearson chi2:                 6.03e+03
No. Iterations:                     7                                         
Covariance Type:            nonrobust                                         
                                           coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------


In [57]:
def checkVIF(X):
    vif = pd.DataFrame()
    vif['Features'] = X.columns
    vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    vif['VIF'] = round(vif['VIF'], 2)
    vif = vif.sort_values(by = "VIF", ascending = False)
    return(vif)

#Calculating the Variance Inflation Factor
# from statsmodels.stats.outliers_influence import variance_inflation_factor
# checkVIF(train_x)

In [58]:
checkVIF(train_x)

C:\Users\sarth\anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


,Features,VIF
8,InternetService_No,inf
9,OnlineBackup_No internet service,inf
10,DeviceProtection_No internet service,inf
0,tenure,14.89
2,TotalCharges,14.23
1,PhoneService,6.87
7,InternetService_Fiber optic,3.93
4,Contract_Two year,3.40
6,MultipleLines_Yes,2.50
11,TechSupport_1,2.05


## Another method to check VIF

In [59]:
# # UDF for calculating vif value
# def vif_cal(input_data, dependent_col):
#     vif_df = pd.DataFrame( columns = ['Var', 'Vif'])
#     x_vars=input_data.drop([dependent_col], axis=1)
#     xvar_names=x_vars.columns
#     for i in range(0,xvar_names.shape[0]):
#         y=x_vars[xvar_names[i]] 
#         x=x_vars[xvar_names.drop(xvar_names[i])]
#         rsq=sm.OLS(y,x).fit().rsquared  
#         vif=round(1/(1-rsq),2)
#         vif_df.loc[i] = [xvar_names[i], vif]
#     return vif_df.sort_values(by = 'Vif', axis=0, ascending=False, inplace=False)

## Making predictions from the Logistic model

In [60]:
from sklearn.linear_model import LogisticRegression

logsk= LogisticRegression()
m = logsk.fit(train_x,train_y)

In [61]:
prediction= m.predict_proba(train_x)

In [62]:
future= m.predict_proba(test_x)

In [63]:
future

array([[0.62718849, 0.37281151],
       [0.39803331, 0.60196669],
       [0.80654319, 0.19345681],
       ...,
       [0.56667507, 0.43332493],
       [0.33603825, 0.66396175],
       [0.97407151, 0.02592849]])

In [64]:
fut= pd.DataFrame(future)

In [65]:
fut.rename(columns={0: 'Not_Churn_Probability',1: 'Churn_Probability'}, inplace= True)

In [66]:
fut

,Not_Churn_Probability,Churn_Probability
0,0.627188,0.372812
1,0.398033,0.601967
2,0.806543,0.193457
3,0.608147,0.391853
4,0.923416,0.076584
...,...,...
2105,0.917751,0.082249
2106,0.856579,0.143421
2107,0.566675,0.433325
2108,0.336038,0.663962


In [67]:
df= pd.DataFrame(test_y)

In [68]:
df.head()

,Churn
4930,0
4931,0
4932,0
4933,0
4934,0


In [69]:
fut['Predicted']= fut.Churn_Probability.map(lambda x: 1 if x>0.5 else 0)

In [70]:
fut.head()

,Not_Churn_Probability,Churn_Probability,Predicted
0,0.627188,0.372812,0
1,0.398033,0.601967,1
2,0.806543,0.193457,0
3,0.608147,0.391853,0
4,0.923416,0.076584,0


In [71]:
#Mergign the data sets side by side
x= fut.reset_index(drop=True).merge(df.reset_index(drop=True), left_index=True, right_index=True)

In [72]:
#dropping unnecessary columns
x.drop(columns=['Not_Churn_Probability','Churn_Probability'],inplace= True)

In [73]:
x.dtypes

Predicted    int64
Churn        int64
dtype: object

In [74]:
#taking the result where if both the Predicted and churn columns are same then it returns 1 else it will return 0
x['result'] = x.apply(lambda x : 1 if  x['Predicted'] == x['Churn'] else 0, axis=1)

In [75]:
x.head(50)

,Predicted,Churn,result
0,0,0,1
1,1,0,0
2,0,0,1
3,0,0,1
4,0,0,1
5,0,0,1
6,0,0,1
7,0,0,1
8,1,1,1
9,0,0,1


In [76]:
#finding the accuracy
x['result'].sum()/len(x)*100

79.33649289099526